In [2]:
#Web Scraping y filtro para Minciencias y separacion de productos

In [3]:
#pip install openpyxl

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from scrapy.selector import Selector
import requests
from bs4 import BeautifulSoup
import warnings

In [2]:
warnings.filterwarnings('ignore')
warnings.warn('InsecureRequestWarning')
warnings.warn('Do not show this message')

In [3]:
dfgrupos = pd.read_excel("BDMicroalgas.xlsx", engine='openpyxl') #, sheet_name='Sheet1'
tamaño_dfgrupos =dfgrupos.shape
url= dfgrupos["LINKS"]

In [4]:
url

0     https://scienti.minciencias.gov.co/gruplac/jsp...
1     https://scienti.minciencias.gov.co/gruplac/jsp...
2     https://scienti.minciencias.gov.co/gruplac/jsp...
3     https://scienti.minciencias.gov.co/gruplac/jsp...
4     https://scienti.minciencias.gov.co/gruplac/jsp...
                            ...                        
94    https://scienti.minciencias.gov.co/gruplac/jsp...
95    https://scienti.minciencias.gov.co/gruplac/jsp...
96    https://scienti.minciencias.gov.co/gruplac/jsp...
97    https://scienti.minciencias.gov.co/gruplac/jsp...
98    https://scienti.minciencias.gov.co/gruplac/jsp...
Name: LINKS, Length: 99, dtype: object

In [5]:
#Para el ejemplo 
#url = url[0:5]

In [5]:
#dividir en varias pasadas

In [ ]:
dfcon = pd.DataFrame()

In [13]:
df = pd.read_excel("InormacionFiltrada.xlsx",engine='openpyxl') #sheet_name='Sheet1', 
df = df.set_index("Index")

In [15]:
#hasta 1 
df

,Nombre del Grupo,Datos básicos,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo,PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN,Programa académico de doctorado,Programa académico de maestría,Otro programa académico,...,Participación Ciudadana en Proyectos de CTI,"Producción en arte, arquitectura y diseño",ACTIVIDADES DE FORMACIÓN,Asesorías al Programa Ondas,Curso de Corta Duración Dictados,Trabajos dirigidos/turorías,ACTIVIDADES COMO EVALUADOR,Jurado/Comisiones evaluadoras de trabajo de grado,Participación en comités de evaluación,Demás trabajos
Index,,,,,,,,,,,,,,,,,,,,,
NaN,Grupo de Investigación en Procesos Avanzados p...,Año y mes de formación: 1998 - 1... \n Depar...,1.- UNIVERSIDAD DEL VALLE - (Avalado) \n 2.- U...,Plan de trabajo: Durante los 15 años de existe...,1.- Desarrollo de procesos y equipos para la r...,Nombre \n 1.- Fiderman Machuca Martínez \n 2.-...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,CENTRO DE INVESTIGACION PARA EL DESARROLLO SOS...,Año y mes de formación: 1993 - 2... \n Depar...,1.- UNIVERSIDAD INDUSTRIAL DE SANTANDER - (Ava...,Plan de trabajo: El Centro de Investigación pa...,1.- Biocombustibles y co-productos a partir de...,Nombre \n 1.- Viatcheslav Kafarov \n 2.- Aliri...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
palabras_filtro = "microalga"
palabras_filtro2 = "tratamiento de agua"

for i in range (2,3): #tamaño_dfgrupos [0]
    count = i
    try:
        print("Procesando link  ",count," .")
        url = dfgrupos["LINKS"][i]
        r = requests.get(url, verify=False)
        soup = BeautifulSoup(r.text, 'lxml')
        NomGrupo = soup.find("span", class_='celdaEncabezado')
        NombreGrupo = NomGrupo.text
        #Filtrar todas las tablas
        table = soup.find_all('table')
        df = pd.read_html(str(table))
        #Numero de de tablas 
        NumTablas = len(df)
        #CICLO 
        dfcon1 = pd.DataFrame()
        dfcon1["Index"] = [i]
        dfcon1["Nombre del Grupo"] = [NombreGrupo]
        #Tabla numero 0, datos basicos, que es diferente 
        #Tabla de datos 
        df = pd.read_html(str(table))[0]
        #Imputacion de los NAN
        df= df.fillna('Sin dato')
        #Tabla 1 
        tamaño = df.shape
        Nombre = df[0][0]
        Dato = []
        for j in range(1,tamaño[0]):
            Dato.append(df[0][j]+":  "+df[1][j]+"... ")
        Datos = " \n ".join(Dato)
        Datos
        dfcon1[Nombre]= [Datos]
        # Siguientes Tablas 1-5
        print("Procesando link  ",count," ..")
        for i in range (1,6):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]):
                Datos.append(df[0][j])
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos] 
        print("Procesando link  ",count," ...")
        # Siguientes Tablas 6-12
        for i in range (6,13):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            df_filtrado = df[df[0].str.contains(palabras_filtro,case=False) & df[0].str.contains(palabras_filtro2,case=False) ] 
            Datos = []
            for j in range(0,tamaño[0]):
                try:
                    Datos.append(df_filtrado[0][j])
                except:
                    pass
            Datos = " \n ".join(Datos) 
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," ....")
        #Tabla 13 -44
        for i in range (13,45):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]):
                try:
                    df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False) ]               
                    Datos.append(df_filtrado[1][j])
                except:
                    pass
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," .....")
        #Table 45 - 46
        for i in range (45,47):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]-1):
                df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False) ]
                Datos.append(df[1][j])
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," ......")
        #Tabla 47 - 84 
        for i in range (47,84):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]-1):
                try:
                    df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False)] #Case = false para que no importen mayusculas
                    Datos.append(df_filtrado[1][j])
                except:
                    pass
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos] 
        dfcon = pd.concat([dfcon, dfcon1])
    except:
            print ("error en link  ",count)
#dfcon = dfcon.set_index("Index")
dfcon

Procesando link   2  .
Procesando link   2  ..
Procesando link   2  ...
Procesando link   2  ....
Procesando link   2  .....
Procesando link   2  ......


,Nombre del Grupo,Datos básicos,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo,PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN,Programa académico de doctorado,Programa académico de maestría,Otro programa académico,...,"Producción en arte, arquitectura y diseño",ACTIVIDADES DE FORMACIÓN,Asesorías al Programa Ondas,Curso de Corta Duración Dictados,Trabajos dirigidos/turorías,ACTIVIDADES COMO EVALUADOR,Jurado/Comisiones evaluadoras de trabajo de grado,Participación en comités de evaluación,Demás trabajos,Index
NaN,Grupo de Investigación en Procesos Avanzados p...,Año y mes de formación: 1998 - 1... \n Depar...,1.- UNIVERSIDAD DEL VALLE - (Avalado) \n 2.- U...,Plan de trabajo: Durante los 15 años de existe...,1.- Desarrollo de procesos y equipos para la r...,Nombre \n 1.- Fiderman Machuca Martínez \n 2.-...,,,,,...,,,,,,,,,,NaN
1.0,CENTRO DE INVESTIGACION PARA EL DESARROLLO SOS...,Año y mes de formación: 1993 - 2... \n Depar...,1.- UNIVERSIDAD INDUSTRIAL DE SANTANDER - (Ava...,Plan de trabajo: El Centro de Investigación pa...,1.- Biocombustibles y co-productos a partir de...,Nombre \n 1.- Viatcheslav Kafarov \n 2.- Aliri...,,,,,...,,,,,,,,,,NaN
0.0,Solidos Porosos y Calorimetría Aplicada,Año y mes de formación: 2000 - 4... \n Depar...,1.- UNIVERSIDAD DE LOS ANDES - (Avalado),Plan de trabajo: El Grupo tiene como plan de a...,1.- DESARROLLO DE CATALIZADORES PARA SINTESIS ...,Nombre \n 1.- Juan Carlos Moreno Piraján \n 2....,,,,,...,,,,,,,,,,2.0


In [12]:
dfcon.to_excel("InormacionFiltrada.xlsx")

In [17]:
dfcon

,Nombre del Grupo,Datos básicos,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo,PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN,Programa académico de doctorado,Programa académico de maestría,Otro programa académico,...,"Producción en arte, arquitectura y diseño",ACTIVIDADES DE FORMACIÓN,Asesorías al Programa Ondas,Curso de Corta Duración Dictados,Trabajos dirigidos/turorías,ACTIVIDADES COMO EVALUADOR,Jurado/Comisiones evaluadoras de trabajo de grado,Participación en comités de evaluación,Demás trabajos,Index
NaN,Grupo de Investigación en Procesos Avanzados p...,Año y mes de formación: 1998 - 1... \n Depar...,1.- UNIVERSIDAD DEL VALLE - (Avalado) \n 2.- U...,Plan de trabajo: Durante los 15 años de existe...,1.- Desarrollo de procesos y equipos para la r...,Nombre \n 1.- Fiderman Machuca Martínez \n 2.-...,,,,,...,,,,,,,,,,NaN
1.0,CENTRO DE INVESTIGACION PARA EL DESARROLLO SOS...,Año y mes de formación: 1993 - 2... \n Depar...,1.- UNIVERSIDAD INDUSTRIAL DE SANTANDER - (Ava...,Plan de trabajo: El Centro de Investigación pa...,1.- Biocombustibles y co-productos a partir de...,Nombre \n 1.- Viatcheslav Kafarov \n 2.- Aliri...,,,,,...,,,,,,,,,,NaN
0.0,Solidos Porosos y Calorimetría Aplicada,Año y mes de formación: 2000 - 4... \n Depar...,1.- UNIVERSIDAD DE LOS ANDES - (Avalado),Plan de trabajo: El Grupo tiene como plan de a...,1.- DESARROLLO DE CATALIZADORES PARA SINTESIS ...,Nombre \n 1.- Juan Carlos Moreno Piraján \n 2....,,,,,...,,,,,,,,,,2.0


In [15]:
display(dfcon.loc[0][50:85])

Empresas de base tecnológica                                                                                                              
APROPIACIÓN SOCIAL Y DIVULGACIÓN PÚBLICA DE LA CIENCIA                                                                                    
Publicaciones editoriales no especializadas                                                                                               
Producciones de contenido digital - Audiovisual                                                                                           
Producciones de contenido digital - Sonoro                                                                                                
Producciones de contenido digital - Recursos gráficos                                                                                     
Divulgación Pública de la Ciencia producción de estrategias y contenidos transmedia                                                       
Desarrollo web             

In [16]:
df = pd.read_excel("InormacionFiltrada.xlsx",engine='openpyxl') #sheet_name='Sheet1', 
df = df.set_index("Index")

In [17]:
df

,Nombre del Grupo,Datos básicos,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo,PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN,Programa académico de doctorado,Programa académico de maestría,Otro programa académico,...,Participación Ciudadana en Proyectos de CTI,"Producción en arte, arquitectura y diseño",ACTIVIDADES DE FORMACIÓN,Asesorías al Programa Ondas,Curso de Corta Duración Dictados,Trabajos dirigidos/turorías,ACTIVIDADES COMO EVALUADOR,Jurado/Comisiones evaluadoras de trabajo de grado,Participación en comités de evaluación,Demás trabajos
Index,,,,,,,,,,,,,,,,,,,,,
0,GRUPO DE INVESTIGACIÓN EN PROCESOS INDUSTRIALE...,Año y mes de formación: 2014 - 1... \n Depar...,1.- Institución Universitaria de Barranquilla ...,Plan de trabajo: El Grupo de investigación en ...,"1.- Diseño de estructuras, mecanismos, máquina...",Nombre \n 1.- Gilberto González García \n 2.- ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grupo de Investigación en Manejo y Agroindustr...,Año y mes de formación: 2007 - 5... \n Depar...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: Los integrantes del grupo de ...,1.- GESTIÓN Y PROCEDIMIENTOS DE CALIDAD DE PRO...,Nombre \n 1.- Saul Dussán Sarria \n 2.- José I...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GNOCIX - Gestión del Conocimiento y la Informa...,Año y mes de formación: 2006 - 3... \n Depar...,1.- Universidad del Sinú - Elías Bechara Zainu...,Plan de trabajo: UNIVERSIDAD DEL SINÚ PLAN DE ...,1.- CIENCIAS BASICAS Y APLICADAS \n 2.- DESARR...,Nombre \n 1.- JUAN MANUEL TORRES TOVIO \n 2.- ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ciencia y Tecnología de Biomoléculas de Interé...,Año y mes de formación: 2003 - 7... \n Depar...,1.- UNIVERSIDAD DEL CAUCA - (Avalado),Plan de trabajo: - Caracterizar diferentes alm...,1.- Caracterización y desarrollo de nuevos pro...,Nombre \n 1.- Héctor Samuel Villada Castillo \...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grupo de Investigación y Desarrollo de Ingenie...,Año y mes de formación: 2004 - 9... \n Depar...,1.- UNIVERSIDAD SANTO TOMAS SECCIONAL TUNJA - ...,Plan de trabajo: Grupo de Investigación y Desa...,1.- Diseño de Maquinaria y Equipo \n 2.- Diseñ...,Nombre \n 1.- Cesar Mauricio Galarza Bogotá \n...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cadenas de Valor,Año y mes de formación: 2007 - 5... \n Depar...,1.- CORPORACIÓN UNIVERSITARIA COMFACAUCA - UNI...,Plan de trabajo: El grupo de investigación Cad...,1.- Produccion y Transformacion Agropecuaria S...,Nombre \n 1.- Zuly Yuliana Delgado ESPINOSA \n...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GIMSC (Grupo de Investigacion en Modelamiento ...,Año y mes de formación: 2002 - 12... \n Depa...,1.- Universidad De San Buenaventura - (Avalado),Plan de trabajo: PLAN DE TRABAJO 1. Estrategia...,1.- Línea De Investigación En Producción De Au...,Nombre \n 1.- William Fernando Espinosa García...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#leer el DataFrame e imprimir cada columna como una tabla nueva

In [18]:
##Eliminar los sin dato y los vacios (:  -)

#Crear tabla de datos basicos
DatosBasicos = pd.DataFrame(columns=['Nombre del Grupo','Datos básicos'])
for j in range(0,df.shape[0]):
    dfDatosBasidos = pd.DataFrame(columns=['Nombre del Grupo','Datos básicos'])
    ListaDatosBasicos = df["Datos básicos"][j].split("... ")
    for i in range (0,len(ListaDatosBasicos)-1):
        if i == 0:
            dfDatosBasidos.loc[len(dfDatosBasidos)] = [df["Nombre del Grupo"][j], ListaDatosBasicos[i]]
        else:
            dfDatosBasidos.loc[len(dfDatosBasidos)] = ["", ListaDatosBasicos[i]]
    #REMOVER LAS FILAS QUE NO SIRVEN 
    v1 = dfDatosBasidos.iloc[:, 1].str.contains("sin dato",case=False)
    v2 = dfDatosBasidos.iloc[:, 1].str.contains(":  -",case=False)
    for i in range (0,len(ListaDatosBasicos)-1):
        if (v1[i] == True) or (v2[i] == True):
            dfDatosBasidos = dfDatosBasidos.drop([i],axis=0)
    DatosBasicos = pd.concat([DatosBasicos, dfDatosBasidos])
DatosBasicos.to_excel("DatosBasicos.xlsx")
DatosBasicos

,Nombre del Grupo,Datos básicos
0,GRUPO DE INVESTIGACIÓN EN PROCESOS INDUSTRIALE...,Año y mes de formación: 2014 - 1
1,,\n Departamento - Ciudad: ATLÁNTICO - SOLEDAD
2,,\n Líder: Gilberto González García
3,,\n ¿La información de este grupo se ha certif...
5,,\n E-mail: gigonzalez@itsa.edu.co
...,...,...
5,,\n E-mail: investiga.ingenieria@usbmed.edu.co
6,,\n Clasificación: A1 con vigencia hasta la ...
7,,\n Área de conocimiento: Ciencias Naturales ...
8,,\n Programa nacional de ciencia y tecnología:...


In [19]:
# Crear tabla de informacion de los grupos 
DatosGrupo = df.iloc[:,0:6].drop(["Datos básicos"],axis=1)
DatosGrupo.to_excel("DatosGrupo.xlsx")
DatosGrupo

,Nombre del Grupo,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo
Index,,,,,
0,GRUPO DE INVESTIGACIÓN EN PROCESOS INDUSTRIALE...,1.- Institución Universitaria de Barranquilla ...,Plan de trabajo: El Grupo de investigación en ...,"1.- Diseño de estructuras, mecanismos, máquina...",Nombre \n 1.- Gilberto González García \n 2.- ...
1,Grupo de Investigación en Manejo y Agroindustr...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: Los integrantes del grupo de ...,1.- GESTIÓN Y PROCEDIMIENTOS DE CALIDAD DE PRO...,Nombre \n 1.- Saul Dussán Sarria \n 2.- José I...
2,GNOCIX - Gestión del Conocimiento y la Informa...,1.- Universidad del Sinú - Elías Bechara Zainu...,Plan de trabajo: UNIVERSIDAD DEL SINÚ PLAN DE ...,1.- CIENCIAS BASICAS Y APLICADAS \n 2.- DESARR...,Nombre \n 1.- JUAN MANUEL TORRES TOVIO \n 2.- ...
3,Ciencia y Tecnología de Biomoléculas de Interé...,1.- UNIVERSIDAD DEL CAUCA - (Avalado),Plan de trabajo: - Caracterizar diferentes alm...,1.- Caracterización y desarrollo de nuevos pro...,Nombre \n 1.- Héctor Samuel Villada Castillo \...
4,Grupo de Investigación y Desarrollo de Ingenie...,1.- UNIVERSIDAD SANTO TOMAS SECCIONAL TUNJA - ...,Plan de trabajo: Grupo de Investigación y Desa...,1.- Diseño de Maquinaria y Equipo \n 2.- Diseñ...,Nombre \n 1.- Cesar Mauricio Galarza Bogotá \n...
5,Cadenas de Valor,1.- CORPORACIÓN UNIVERSITARIA COMFACAUCA - UNI...,Plan de trabajo: El grupo de investigación Cad...,1.- Produccion y Transformacion Agropecuaria S...,Nombre \n 1.- Zuly Yuliana Delgado ESPINOSA \n...
6,GIMSC (Grupo de Investigacion en Modelamiento ...,1.- Universidad De San Buenaventura - (Avalado),Plan de trabajo: PLAN DE TRABAJO 1. Estrategia...,1.- Línea De Investigación En Producción De Au...,Nombre \n 1.- William Fernando Espinosa García...


In [20]:
# Crear todas las tablas en archivos diferentes. 

df1 = df.set_index("Nombre del Grupo")
for i in range(5,84):
    try:
        NombreColumna = df1.columns[i]
        Nombre = str(i)+' '+NombreColumna+".xlsx"
        dfsin_nan = df1[NombreColumna].dropna()
        try:
            if type(dfsin_nan[0]) != str():
                dfsin_nan.to_excel(Nombre)
        except:
            pass
    except:
        pass